Setup:

In [ ]:
#!sudo apt install -qq -y fluidsynth >/dev/null;  # that's for Linux.  on Mac: "brew install fluidsynth"
#!pip install -Uqq pyfluidsynth pretty_midi wandb

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from glob import glob
import pathlib
import pretty_midi
from tqdm import tqdm_notebook as tqdm
import multiprocessing as mp
from tqdm.contrib.concurrent import process_map
import random

In [ ]:
#!wget -N https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip
#!unzip -n -qq maestro-v2.0.0-midi.zip

In [ ]:
#data_dir = pathlib.Path('maestro-v2.0.0')
#filenames = glob(str(data_dir/'**/*.mid*'))
#print('Number of files:', len(filenames))

In [ ]:
vocab_size=128 # number of midi notes
n_contin = 2   # number of continuous variables besides pitches

def midi_file_to_tensor(midi_file):
    pm = pretty_midi.PrettyMIDI(midi_file) # read in the whole file. this is incredibly slow

    # Sort the notes first by start time (then by pitch if two notes start at the same time)
    sorted_notes = sorted(pm.instruments[0].notes, key=lambda note: (note.start, note.pitch))
    notes = torch.empty( (len(sorted_notes), 3), dtype=torch.float32 ) # allocate storage

    prev_start = sorted_notes[0].start
    for i, note in enumerate(sorted_notes):
        notes[i] = note.pitch
        notes[i, 1] = note.start - prev_start  # step, i.e. time since last note started
        notes[i, 2] = note.end - note.start    # duration
        prev_start = note.start

    return notes

In [ ]:
#notes = midi_file_to_tensor(filenames[0])
#pitches = notes[:,0].type(torch.long)  # just the pitch info
#notes.shape, pitches.shape

In [ ]:
# @title Tensor to MIDI Display Code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional
from IPython.display import Audio, display


def notes_arr_to_df(notes_arr) -> pd.DataFrame:
    columns = ['pitch','step','duration']
    df = pd.DataFrame(notes_arr, columns=columns)
    df["start"] = ""
    df["end"] = ""

    prev_start = 0
    #for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    for i, row in df.iterrows():
        start = prev_start + float(row['step'])
        df.at[i, 'start'] = start
        df.at[i, 'end'] = start + float(row['duration'])
        prev_start = start
    return df

def df_to_midi(
        notes_df: pd.DataFrame,
        out_file: str = '',  # output file to save to, if any
        instrument_name: str = 'Acoustic Grand Piano', # whatever you want to call this instrument
        velocity: int = 100,  # note loudness
    ) -> pretty_midi.PrettyMIDI:
    "converts a dataframe to valid midi"

    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(
        program=pretty_midi.instrument_name_to_program(
            instrument_name))

    prev_start = 0
    for i, note in notes_df.iterrows(): # this is a serial operation, not sure how to parallelize
        start = float(prev_start + note['step'])
        end = float(start + note['duration'])
        note = pretty_midi.Note(
            velocity=velocity,
            pitch=int(note['pitch']),
            start=start,
            end=end,
        )
        instrument.notes.append(note)
        prev_start = start

    pm.instruments.append(instrument)
    if out_file: pm.write(out_file)
    return pm

def plot_piano_roll(notes_df: pd.DataFrame, count: Optional[int] = None):
    "produce a piano roll plot"
    if count:
        title = f'First {count} notes'
    else:
        title = f'Whole track'
        count = len(notes_df['pitch'])
    plt.figure(figsize=(20, 4))
    plot_pitch = np.stack([notes_df['pitch'], notes_df['pitch']], axis=0)
    plot_start_stop = np.stack([notes_df['start'], notes_df['end']], axis=0)
    plt.plot(
        plot_start_stop[:, :count], plot_pitch[:, :count], color="b", marker=".")
    plt.xlabel('Time [s]')
    plt.ylabel('Pitch')
    ax = plt.gca()
    ax.set_ylim([0, vocab_size])
    _ = plt.title(title)
    plt.show()


def midi_to_audio(pm: pretty_midi.PrettyMIDI, seconds=30, sr=16000):
  "midi to audio, playable in notebook"
  waveform = pm.fluidsynth(fs=float(sr))
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*sr]
  return display(Audio(waveform_short, rate=sr))

def pitches_to_midi(pitch_list, seconds=30):
    notes_tensor = torch.zeros((len(pitch_list), 3)) + 0.25
    for i, p in enumerate(pitch_list):
        notes_tensor[i,0] = p
    notes_df = notes_arr_to_df(notes_tensor.cpu().detach().numpy())
    midi = df_to_midi(notes_df)
    plot_piano_roll(notes_df)
    audio_display = midi_to_audio(midi, seconds=seconds)
    return audio_display

def notes_to_midi(notes_tensor, seconds=30):
    #notes_tensor = notes_tensor * (notes_tensor>0)  # negative numbers clipped to zero
    if notes_tensor.min() < 0.0:
      print("WARNING: You have negative pitches, steps or durations. Setting them to zero")
      notes_tensor = notes_tensor * (notes_tensor >= 0)
    notes_df = notes_arr_to_df(notes_tensor.cpu().detach().numpy())
    midi = df_to_midi(notes_df)
    plot_piano_roll(notes_df)
    audio_display = midi_to_audio(midi, seconds=seconds)
    return audio_display

In [ ]:
def old_files_to_tensor_list(filenames, n_files=None): # takes ~2.5 seconds per file
  tensor_list = []
  if n_files is None: n_files = len(filenames)
  for f in tqdm(filenames[:n_files]):
    tensor_list.append(midi_file_to_tensor(f))
  return tensor_list


def files_to_tensor_list(filenames): # reads files in parallel! Take 2 mins 30s intstead of 20 mins
    tensor_list = process_map(midi_file_to_tensor, filenames, max_workers=mp.cpu_count(), chunksize=1)
    return tensor_list

In [ ]:
#notes_list = files_to_tensor_list(filenames, n_files=len(filenames))
#print(f"\n{len(notes_list)} files read")

In [ ]:
notes_list = torch.load('maestro3_tensor_list.pt')
len(notes_list)

In [ ]:
use_wandb = True
if use_wandb:
    import wandb
    wandb.login()

In [ ]:
device = 'cuda:1' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print("device =",device)

# Hyperparameters

In [ ]:

# hyperparameters
learning_rate = 1e-3
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
n_embd = 256# 64  # dimensions of embeddings
n_head = 16# 4
n_layer = 16# 4
dropout = 0.1 #0.1

max_iters = 100000
eval_interval = 100
eval_iters = 200
# ------------

config = {
  'learning_rate': learning_rate,
  'batch_size': batch_size,
  'block_size': block_size,
  'n_embd': n_embd,
  'n_head': n_head,
  'n_layer': n_layer,
  'dropout': dropout
}

Dataset creation

In [ ]:
def tl_to_notes(tensor_list, shuffle=False, delimit=True):
  "list of tensors (of arbitrary length, for each song) converted to one big long tensor of notes all running togehter"
  if shuffle:random.shuffle(tensor_list)
  if delimit:
    delimiter = torch.zeros(3)  # use all zeros to show ends of songs
    tensor_list = [element for item in tensor_list for element in (item, delimiter)]
  return torch.vstack(tensor_list)

seed = 1337
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

all_notes = tl_to_notes(notes_list, shuffle=True) # just grab one file, for testing overfitting
#all_notes = tl_to_notes(notes_list, shuffle=True)     # all the songs you read in earlier
all_pitches = all_notes[:,0].type(torch.long)  # just the pitch info
print("all_notes.shape =",all_notes.shape)

data = all_notes # all_notes
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
print("len(train_data), len(val_data) =",len(train_data), len(val_data) )
#assert n_contin == train_data.shape[-1]-1  # data consists of pitches plus continuous variables


def augment_data(x,y):
    pitch_shift = torch.randint(low=-12, high=12, size=(1,1,1), dtype=torch.long).item()
    time_scale = 0.5+torch.rand((1,1,1),dtype=torch.float32).item()  # [0.5 .. 1.5]

    x[:,:,0] = torch.clamp( x[:,:,0]+pitch_shift, min=0, max=vocab_size)
    y[:,:,0] = torch.clamp( y[:,:,0]+pitch_shift, min=0, max=vocab_size)
    x[:,:,1:] = x[:,:,1:] * time_scale
    y[:,:,1:] = y[:,:,1:] * time_scale
    return x, y


def get_batch(split, debug=False):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.clone(), y.clone()  # just to avoid overwrites
    if split == 'train': # data augmentation
        x, y = augment_data(x,y)
    x[:,0,1] = x[:,0,1] * 0  # set first step of every x sequence to 0 (this shouldn't make a difference though)
    x, y = x.to(device), y.to(device)
    return x, y
    

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        pitch_losses = torch.zeros(eval_iters)
        contin_losses = torch.zeros(eval_iters)

        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, contin_vars, loss, sublosses = model(X, Y)
            losses[k] = loss.item()
            pitch_losses[k] = sublosses['pitch']
            contin_losses[k] = sublosses['contin']
        out[split] = losses.mean()
        out['pitch'] = pitch_losses.mean()
        out['contin'] = contin_losses.mean()
    model.train()
    return out


In [ ]:
x,y = get_batch('train',debug=True)
print(f"B, T = {batch_size}, {block_size}")

x is a sequence

In [ ]:
notes_to_midi(x[0])
x[0][:10]

y is x shifted back by one and including new data.
in this sense only y[:,-1] is the "next token" being predicted.

In [ ]:
notes_to_midi(y[0])
y[0][:10]

Model definition

In [ ]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


def mse_with_positive_pressure(y_true: torch.Tensor, y_pred: torch.Tensor):
  # from Magenta example, converted to pytorch
  se = (y_true - y_pred) ** 2
  positive_pressure = 10 * torch.clamp(-y_pred, min=0) # ten times the negative values made positive
  return (se + positive_pressure).mean()


# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd-n_contin)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)
        self.contin_head = nn.Linear(n_embd, n_contin, bias=False)
        
    def forward(self, inputs, targets=None):
        idx = inputs[:,:,0].type(torch.long).to(inputs.device)  # idx are the pitch indices
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C-2) # embed the pitches
        tok_emb = torch.cat((tok_emb, inputs[:,:,1:]), dim=-1)   # concat the continuous variables
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x)   # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)
        contin_vars = self.contin_head(x) # (B,T,2)
        #contin_vars = F.softplus(contin_vars) # just to prevent negative values
        time_step = F.leaky_relu(contin_vars[:,:,0]).unsqueeze(-1)  # encourage non-negative steps, but allow zero and small neg values
        time_dur = F.softplus(contin_vars[:,:,1]).unsqueeze(-1)     # force only positive durations.
        contin_vars = torch.cat( (time_step, time_dur), dim=-1 )
        
        # we can also compute losses if targets are included in .forward call
        if targets is None:
            return logits, contin_vars, None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            pitch_targets = (targets[:,:,0]).type(torch.long).to(logits.device).view(B*T)
            
            lambda_pitch, lambda_contin = 0.5, 10.0  # values from Magenta example, rescaled
            #lambda_pitch, lambda_contin = 0.25, 20.0  # half pitch, double contin
            pitch_loss = lambda_pitch * F.cross_entropy(logits, pitch_targets)

            # contin variables, use mse with postive pressure
            contin_loss = lambda_contin * F.mse_loss(contin_vars, targets[:,:,-n_contin:] )
            #contin_loss = lambda_contin * mse_with_positive_pressure(contin_vars, targets[:,:,-n_contin:] )
            sublosses = {'pitch':pitch_loss.item(), 'contin':contin_loss.item() } 

            loss = pitch_loss + contin_loss
            return logits, contin_vars, loss, sublosses

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0):
        # idx is (B, T, 1+contin_vars) array of values in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, contin_vars, loss = self(idx_cond)
            # focus only on the last time step
            logits, contin_vars = logits[:, -1, :], contin_vars[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits/temperature, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # concat continuous variables
            idx_next = torch.cat((idx_next, contin_vars), dim=-1).unsqueeze(0) # unsqueeze to add dummy batch dim
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx.cpu()



Instantiate and get ready to run

In [ ]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate/50)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 0.01, total_steps=max_iters)

In [ ]:
def save_checkpoint(step, model, optimizer,loss, name):
    name = name + f'_{step}.pt'
    print("Saving checkpoint to",name)
    torch.save({
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, name)

def load_checkpoint(name, model, optimizer):
    checkpoint = torch.load('augemented_cuda_checkpoint.pt')
    model.load_state_dict(checkpoint['model_state_dict'])
    m = model.to(device)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    step = checkpoint['step']
    loss = checkpoint['loss']
    return step, m, optimizer, loss

In [ ]:
load_the_checkpoint = True

if load_the_checkpoint:
    step, model, optimizer, loss = load_checkpoint('giant_model_lrelu_checkpoint_24000.pt', model, optimizer)

In [ ]:
if use_wandb: wandb.init(project='musicbox', config=config)

Do training

In [ ]:
loss_hist = []
checkpoint_every = 1000
for step in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if step % eval_interval == 0 or step == max_iters - 1:
        losses = estimate_loss()
        print(f"step {step}: losses: train {losses['train']:.4f}, val {losses['val']:.4f}, pitch {losses['pitch']:.4f}, contin {losses['contin']:.4f}")
        if use_wandb: wandb.log(losses | {'step':(step+19000)//eval_interval})

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, contin_vars, loss, sublosses = model(xb, yb)

    if (step>0) and (step % checkpoint_every==0):
        save_checkpoint(step, model, optimizer, loss, "giant_model_lrelu_checkpoint")
        
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    scheduler.step()

In [ ]:
#torch.save({
#            'step': 10000,
#            'model_state_dict': model.state_dict(),
#            'optimizer_state_dict': optimizer.state_dict(),
#            'loss': loss,
#            }, f'giant_model_lrelu_checkpoint_{step}.pt')

save_checkpoint(step, model,optimizer,loss, "giant_model_lrelu_checkpoint")

In [ ]:
if use_wandb: wandb.finish()

# Evaluate Generative Capabilities

In [ ]:
# input example (from training dataset)
input_example = notes_list[0]
print("len(input_example) =", len(input_example))
notes_to_midi(input_example)

In [ ]:
n_starting_notes = 24  # how many real notes to start with
starting_notes = input_example[:n_starting_notes]
#notes_to_midi(starting_notes)

Generate

In [ ]:
# generate output variations
context = starting_notes.detach().to(device=device).unsqueeze(0)

for variation in range(1):
    notes_gen = m.generate(context, max_new_tokens=80, temperature=1)[0]
    print(len(input_example), len(notes_gen))
    notes_to_midi(notes_gen)

Links:

https://archives.ismir.net/ismir2021/latebreaking/000005.pdf

https://dl.acm.org/doi/10.1145/3394171.3413671

https://miditok.readthedocs.io/en/latest/index.html

https://github.com/lucasnfe/adl-piano-midi

https://medium.com/mlearning-ai/generating-music-with-gpt-b0f4ab738b58

https://arxiv.org/pdf/1809.04281.pdf

https://magenta.tensorflow.org/music-transformer

https://colab.research.google.com/notebooks/magenta/piano_transformer/piano_transformer.ipynb

https://github.com/gwinndr/MusicTransformer-Pytorch

# Investigating Data Distributions

In [ ]:
data.shape

In [ ]:
notes_list[0][:,2].max()

In [ ]:
type(1) == int

In [ ]:
bins=400

def plot_hist(songs, bins=None, max_time=None):
    fig, ax = plt.subplots(1,3, figsize=(12,4))
    ax[0].set_ylabel("Count")
    if type(songs[0]) == int: 
        songs = [notes_list[s] for s in songs]
    for s_ind, song in enumerate(songs):
        for i in range(len(ax)):
            ax[i].set_xlabel(f"{['Notes','Steps','Durations'][i]}")
            data = song[:,i]
            minval, maxval =  data.min().item(), data.max().item()
            if i > 0 and max_time is not None: maxval = max_time
            bins = int(len(data))//4 if bins is None else bins
            hist = np.array(torch.histc(data, bins=bins, min=minval, max=maxval).tolist())
            t = np.linspace(minval, maxval, num=bins)
            modes = np.argpartition(hist, -2)[-2:]
            mode = modes[(i+1) % 2]
            mode = minval + mode*(maxval-minval)/bins
            
            print("s_ind, i, minval, maxval, bins, mode = ",s_ind, i, minval, maxval, bins, mode)

            #print("i, maxval, bins = ",i, maxval, bins)
            ax[i].bar(t, hist, align='center', width=(maxval-minval)/bins, alpha=0.5)
    plt.show()

for i in [0,1,2,3]:
    notes_to_midi(notes_list[i])
    plot_hist([i], bins=32, max_time=2.0)

In [ ]:
notes_to_midi(input_example)
plot_hist([input_example], bins=32) 

In [ ]:
notes_to_midi(notes_gen)
plot_hist([notes_gen], bins=32) 

In [ ]:
plot_hist([all_notes], bins=32, max_time=2.0) 

In [ ]:
all_pitches.min()